#### Setup and parameters

In [1]:
import warnings
warnings.simplefilter('ignore')
import deeplabcut
import pandas as pd
import ruamel.yaml
import sys
import os
import tensorflow as tf
import numpy as np
import re

path_config_file = r"C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-separate\config.yaml"

# wd=r"Z:\lab\NSF forelimb project\Phil_lab\dlc-data-swap"
possum_megavids=[r"Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\11Apr18.LaiRegnault.SEP101.LS.biceps_teres_lat\c1_11Apr.mp4",
               r"Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\11Apr18.LaiRegnault.SEP101.LS.biceps_teres_lat\c2_11Apr.mp4"]
experimenter="Phil"


hdf_path=r"C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-diff\labeled-data\11Apr_diff\machinelabels-iter0.h5"

bodyparts = ['Body_ds1_crn_cam1',
             'Body_ds1_crn_cam2',
             'Body_ds2_int_cam1',
             'Body_ds2_int_cam2',
             'Body_ds3_cdl_cam1',
             'Body_ds3_cdl_cam2',
             'Body_vn1_crn_cam1',
             'Body_vn1_crn_cam2',
             'Body_vn2_int_cam1',
             'Body_vn2_int_cam2',
             'Body_vn3_cdl_cam1',
             'Body_vn3_cdl_cam2',
             'Scapula_acr_cam1',
             'Scapula_acr_cam2',
             'Scapula_spi_cam1',
             'Scapula_spi_cam2',
             'Scapula_vtb_cam1',
             'Scapula_vtb_cam2',
             'Humerus_dpc_cam1',
             'Humerus_dpc_cam2',
             'Humerus_ent_cam1',
             'Humerus_ent_cam2',
             'Humerus_ect_cam1',
             'Humerus_ect_cam2',
             'Ulna_olc_cam1',
             'Ulna_olc_cam2',
             'Ulna_int_cam1',
             'Ulna_int_cam2',
             'Ulna_dst_cam1',
             'Ulna_dst_cam2',
             'Radius_prx_cam1',
             'Radius_prx_cam2',
             'Radius_int_cam1',
             'Radius_int_cam2',
             'Radius_dst_cam1',
             'Radius_dst_cam2',
             'Teres_maj_prx_cam1',
             'Teres_maj_prx_cam2',
             'Teres_maj_dst_cam1',
             'Teres_maj_dst_cam2',
             'Biceps_prx_cam1',
             'Biceps_prx_cam2',
             'Biceps_dst_cam1',
             'Biceps_dst_cam2']

#### Create new project and extract frames

In [2]:
task='possum101_11Apr'
path_config_file=deeplabcut.create_new_project(task,experimenter,possum_megavids, working_directory=wd, videotype='.mp4', copy_videos=False) 
%matplotlib inline
deeplabcut.extract_frames(path_config_file, userfeedback=False) 

Created "\\rcstore02.rc.fas.harvard.edu\spierce_lab\lab\NSF forelimb project\Phil_lab\dlc-data-swap\possum101_11Apr-Phil-2020-04-13\videos"
Created "\\rcstore02.rc.fas.harvard.edu\spierce_lab\lab\NSF forelimb project\Phil_lab\dlc-data-swap\possum101_11Apr-Phil-2020-04-13\labeled-data"
Created "\\rcstore02.rc.fas.harvard.edu\spierce_lab\lab\NSF forelimb project\Phil_lab\dlc-data-swap\possum101_11Apr-Phil-2020-04-13\training-datasets"
Created "\\rcstore02.rc.fas.harvard.edu\spierce_lab\lab\NSF forelimb project\Phil_lab\dlc-data-swap\possum101_11Apr-Phil-2020-04-13\dlc-models"
Creating the symbolic link of the video
Created the symlink of Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\11Apr18.LaiRegnault.SEP101.LS.biceps_teres_lat\c1_11Apr.mp4 to \\rcstore02.rc.fas.harvard.edu\spierce_lab\lab\NSF forelimb project\Phil_lab\dlc-data-swap\possum101_11Apr-Phil-2020-04-13\videos\c1_11Apr.mp4
Created the symlink of Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\11Apr18.LaiRegnault.SEP101.LS.bicep

In [9]:
import os
import re
        
def scanDir(directory, extension='avi', filters=[], filter_out=True, verbose=False):
    file_list=[]
    for root, dirs, files in os.walk(directory):
        for name in files:
            if name.lower().endswith(extension):
                filename = os.path.join(root, name)
                if verbose == True:
                    print("Found file with extension ."+ extension + ": " + filename)
                file_list.append(filename)
                continue
            else:
                continue
    if len(filters) != 0:
        if filter_out==True:
            for string in filters:
                file_list = [file for file in file_list if not re.search(string, file)]
        else:
            for string in filters:
                file_list = [file for file in file_list if re.search(string, file)]
    return(file_list)

In [3]:
import os
import numpy as np
import cv2
        
def vidToPngs(video_path, output_dir=None, indices_to_match=[], name_from_folder=True):
    frame_index = 0
    frame_counter = 0
    if name_from_folder:
        out_name = os.path.splitext(os.path.basename(video_path))[0]
    else:
        out_name = 'img'
    if output_dir==None:
        out_dir = os.path.join(os.path.dirname(video_path),out_name)
    else:
        out_dir = output_dir
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    frame_rate = round(cap.get(5),2)
    while(cap.isOpened()):
        ret, frame = cap.read()
        frame_counter += 1
        if ret == True:
            if indices_to_match and not frame_index in indices_to_match:
                frame_index += 1
                continue
            else:
                png_name = out_name+str(frame_index).zfill(4)+'.png'
                png_path = os.path.join(out_dir, png_name)
                cv2.imshow('frame',frame)
                cv2.imwrite(png_path, frame)
                frame_counter = 0                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                frame_index += 1
        else: 
            break
                
    cap.release()
    cv2.destroyAllWindows()
    print("done!")

In [11]:
import os

def matchFrames(extracted_dir):
    extracted_files = scanDir(extracted_dir, extension='png') 
    extracted_indices = [int(os.path.splitext(os.path.basename(png))[0][3:].lstrip('0')) for png in extracted_files]
    unique_indices = []
    unique_indices = [index for index in extracted_indices if not index in unique_indices]
    result = sorted(unique_indices)
    return result   

def extractMatchedFrames(indices, output_dir = None, src_vids=[]):
    extracted_frames = []
    for video in src_vids:
        out_name = os.path.splitext(os.path.basename(video))[0]+'_matched'
        if output_dir is not None:
            output = output_dir
        else:
            output = os.path.join(os.path.dirname(video),out_name)
        vidToPngs(video, output, indices_to_match=indices, name_from_folder=False)
        extracted_frames.append(output)
    return extracted_frames


In [2]:
import pandas as pd
import os

def spliceXma2Dlc(substitute_video,project_path,csv_path, frame_indices):
    substitute_name = os.path.splitext(os.path.basename(substitute_video))[0]
    substitute_data_relpath = os.path.join("labeled-data",substitute_name)
    substitute_data_abspath = os.path.join(project_path,substitute_data_relpath)

    df=pd.read_csv(csv_path,sep=',',header=0,dtype='float',na_values='NaN')
    names = df.columns.values
    parts = [name.rsplit('_',1)[0] for name in names]
    parts_unique = []
    for part in parts:
        if not part in parts_unique:
            parts_unique.append(part)
    df['frame_index']=[os.path.join(substitute_data_relpath,'img'+str(index).zfill(4)+'.png') for index in frame_indices]
    df['scorer']=experimenter
    df = df.melt(id_vars=['frame_index','scorer'])
    new = df['variable'].str.rsplit("_",n=1,expand=True)
    df['variable'],df['coords'] = new[0], new[1]
    df=df.rename(columns={'variable':'bodyparts'})
    df['coords']=df['coords'].str.rstrip(" ").str.lower()
    cat_type = pd.api.types.CategoricalDtype(categories=parts_unique,ordered=True)
    df['bodyparts']=df['bodyparts'].str.lstrip(" ").astype(cat_type)
    newdf = df.pivot_table(columns=['scorer', 'bodyparts', 'coords'],index='frame_index',values='value',aggfunc='first',dropna=False)
    newdf.index.name=None
    ##export
    if not os.path.exists(substitute_data_abspath):
        os.mkdir(substitute_data_abspath)
    data_name = os.path.join(substitute_data_abspath,("CollectedData_"+experimenter+".h5"))
    newdf.to_hdf(data_name, 'df_with_missing', format='table', mode='w')
    newdf.to_csv(data_name.split('.h5')[0]+'.csv')
    print("saved "+str(data_name))
    return substitute_data_abspath, parts_unique


def splitDlc2Xma(hdf_path, bodyparts):
    bodyparts_XY = []
    for part in bodyparts:
        bodyparts_XY.append(part+'_X')
        bodyparts_XY.append(part+'_Y')
    df=pd.read_hdf(hdf_path)
    # extracted_frames= [index for index in df.index]
    df = df.reset_index().melt(id_vars=['index'])
    df = df[df['coords'] != 'likelihood']
    df['id'] = df['bodyparts']+'_'+df['coords'].str.upper()
    df[['index','value','id']]
    df = df.pivot(index='index',columns='id',values='value')
    extracted_frames = [index.split('\\')[-1] for index in df.index]
    df = df.reindex(columns=bodyparts_XY)
    df.to_csv(os.path.splitext(hdf_path)[0]+'_split'+'.csv',index=False)
    return extracted_frames

splitDlc2Xma(hdf_path, bodyparts)


NameError: name 'hdf_path' is not defined

NameError: name 'hdf_path' is not defined

In [13]:
%%capture
import ruamel.yaml


def updateConfig(path_config_file, videos=[],bodyparts=[],corner2move2=512):
    config = ruamel.yaml.load(open(path_config_file))
    if videos:
        video_sets={video:{"crop":"0, 1024, 0, 1024"} for video in videos}
        config['video_sets']=video_sets
    if bodyparts:
        config['bodyparts']=bodyparts
    config['corner2move2']=[corner2move2,corner2move2]
    ruamel.yaml.round_trip_dump(config, sys.stdout)
    with open(path_config_file, 'w') as fp:
        ruamel.yaml.round_trip_dump(config, fp)
        fp.close()
    return path_config_file


# Digitize extracted frames in xmalab, then take marker IDs from output csv and convert digitized data to h5

In [14]:
substitute_video = r"Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\11Apr18.LaiRegnault.SEP101.LS.biceps_teres_lat\11Apr_black.mp4"



xma_csv_path = r"Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\11Apr18.LaiRegnault.SEP101.LS.biceps_teres_lat\c1and2_11Apr_matched_xmalab2D.csv"
project_path = os.path.dirname(path_config_file)
extracted_dir = os.path.join(project_path,"labeled-data")




##get matching frames from merged video 
frame_indices = matchFrames(extracted_dir)
spliced_folder, bodyparts = spliceXma2Dlc(substitute_video,project_path,xma_csv_path,frame_indices)

extractMatchedFrames(frame_indices, output_dir = spliced_folder, src_vids=[substitute_video])

##then remove separate cameras from project and insert new folder with merged video and extracted frames
updateConfig(path_config_file, videos=[substitute_video],bodyparts=bodyparts)

saved C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-black\labeled-data\11Apr_black\CollectedData_Phil.h5
done!
Task: possum101_11Apr
scorer: Phil
date: Apr13
project_path: C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-black
video_sets:
  Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\11Apr18.LaiRegnault.SEP101.LS.biceps_teres_lat\11Apr_black.mp4:
    crop: 0, 1024, 0, 1024
bodyparts:
- Body_ds1_crn_cam1
- Body_ds1_crn_cam2
- Body_ds2_int_cam1
- Body_ds2_int_cam2
- Body_ds3_cdl_cam1
- Body_ds3_cdl_cam2
- Body_vn1_crn_cam1
- Body_vn1_crn_cam2
- Body_vn2_int_cam1
- Body_vn2_int_cam2
- Body_vn3_cdl_cam1
- Body_vn3_cdl_cam2
- Scapula_acr_cam1
- Scapula_acr_cam2
- Scapula_spi_cam1
- Scapula_spi_cam2
- Scapula_vtb_cam1
- Scapula_vtb_cam2
- Humerus_dpc_cam1
- Humerus_dpc_cam2
- Humerus_ent_cam1
- Humerus_ent_cam2
- Humerus_ect_cam1
- Humerus_ect_cam2
- Ulna_olc_cam1
- Ulna_olc_cam2
- Ulna_int_cam1
- Ulna_int_cam2
- Ulna_dst_cam

'C:\\Users\\LabAdmin\\Documents\\Phil\\DeepLabCut\\dev\\possum101_11Apr-Phil-2020-04-13-black\\config.yaml'

#### Check imported labels

In [15]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Phil.
They are stored in the following folder: C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-black\labeled-data\11Apr_black_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


In [22]:
deeplabcut.__file__


'C:\\Users\\LabAdmin\\.conda\\envs\\dlc-windowsGPU\\lib\\site-packages\\deeplabcut\\__init__.py'

#### 8. Create training set

In [45]:
deeplabcut.create_training_dataset(path_config_file, augmenter_type="imgaug")
trainposeconfigfile,testposeconfigfile,snapshotfolder=deeplabcut.return_train_network_path(path_config_file,1,0.95)
cfg_dlc=deeplabcut.auxiliaryfunctions.read_plainconfig(trainposeconfigfile)

cfg_dlc['scale_jitter_lo']= 0.5
cfg_dlc['scale_jitter_up']=1.5

cfg_dlc['augmentationprobability']=.5
cfg_dlc['batch_size']=1 #pick that as large as your GPU can handle it
cfg_dlc['elastic_transform']=True
cfg_dlc['rotation']=180
cfg_dlc['grayscale']=False
cfg_dlc['hist_eq']=True
cfg_dlc['fliplr']=True
cfg_dlc['covering']=True
cfg_dlc['motion_blur'] = True
cfg_dlc['optimizer'] ="adam"
cfg_dlc['dataset_type']='imgaug'
cfg_dlc['multi_step']=[[1e-4, 7500], [5*1e-5, 12000], [1e-5, 50000], [5e-6, 200000]]

deeplabcut.auxiliaryfunctions.write_plainconfig(trainposeconfigfile,cfg_dlc)

# deeplabcut.create_training_model_comparison(path_config_file,num_shuffles=1,net_types=['resnet_50'],augmenter_types=['default','imgaug'])


# 	print("EVALUATE")
# 	deeplabcut.evaluate_network(path_config_file, Shuffles=[shuffle],plotting=True)

# 	print("Analyze Video")

# 	videofile_path = os.path.join(os.getcwd(),'openfield-Pranav-2018-10-30','videos','m3v1mp4.mp4')

# 	deeplabcut.analyze_videos(path_config_file, [videofile_path], shuffle=shuffle)

# 	print("Create Labeled Video and plot")
# 	deeplabcut.create_labeled_video(path_config_file,[videofile_path], shuffle=shuffle)
# 	deeplabcut.plot_trajectories(path_config_file,[videofile_path], shuffle=shuffle)

C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-AUGdiff\training-datasets\iteration-0\UnaugmentedDataSet_possum101_11AprApr13  already exists!
C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-AUGdiff\dlc-models\iteration-0\possum101_11AprApr13-trainset95shuffle1  already exists!
C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-AUGdiff\dlc-models\iteration-0\possum101_11AprApr13-trainset95shuffle1/train  already exists!
C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-AUGdiff\dlc-models\iteration-0\possum101_11AprApr13-trainset95shuffle1/test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


In [41]:
for shuffle in [0,1]:
	print("TRAIN NETWORK", shuffle)
	deeplabcut.train_network(path_config_file, shuffle=shuffle,saveiters=10000,displayiters=200,maxiters=5,max_snapshots_to_keep=11)

TRAIN NETWORK 0
The training datafile  C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-AUGdiff\dlc-models\iteration-0\possum101_11AprApr13-trainset95shuffle0\train\pose_cfg.yaml  is not present.
Probably, the training dataset for this specific shuffle index was not created.
Try with a different shuffle/trainingsetfraction or use function 'create_training_dataset' to create a new trainingdataset with this shuffle index.


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\LabAdmin\\Documents\\Phil\\DeepLabCut\\dev\\possum101_11Apr-Phil-2020-04-13-AUGdiff\\dlc-models\\iteration-0\\possum101_11AprApr13-trainset95shuffle0\\train'

#### 9. Start training

In [ ]:
deeplabcut.train_network(path_config_file, displayiters=10,saveiters=10000, maxiters=200000)

In [53]:
path_config_file = r"C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-diff\config.yaml"


In [54]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:37<00:00,  1.08it/s]


In [ ]:
deeplabcut.analyze_videos(path_config_file,[r"Z:\lab\NSF forelimb project\Phil_lab\dlc-data-swap\11Apr_diff.mp4"])

In [52]:
deeplabcut.create_labeled_video(path_config_file,[r"Z:\lab\NSF forelimb project\Phil_lab\dlc-data-swap\11Apr_diff.mp4"])

Starting %  Z:\lab\NSF forelimb project\Phil_lab\dlc-data-swap ['Z:\\lab\\NSF forelimb project\\Phil_lab\\dlc-data-swap\\11Apr_diff.mp4']
Loading  Z:\lab\NSF forelimb project\Phil_lab\dlc-data-swap\11Apr_diff.mp4 and data.
7904
Duration of video [s]:  263.47 , recorded with  30.0 fps!
Overall # of frames:  7904 with cropped frame dimensions:  1024 1024
Generating frames and creating video.


100%|█████████████████████████████████████████████████████████████████████████████| 7904/7904 [01:15<00:00, 104.84it/s]


In [ ]:
deeplabcut.extract_outlier_frames(path_config_file,[r"Z:\lab\NSF forelimb project\Phil_lab\dlc-data-swap\11Apr_diff.mp4"],outlieralgorithm='jump', extractionalgorithm='kmeans', automatic=True, epsilon=30, comparisonbodyparts=['Ulna_olc_cam1','Ulna_dst_cam1','Radius_prx_cam1','Radius_dst_cam1','Humerus_ent_cam1','Humerus_ect_cam1'])

In [37]:
def xmalab2dlc(run,csv_path,labeled_data_path, width=1024, h_flip=False):  
    ## import XMAlab 2D exports
    df = pd.read_csv(csv_path, sep=',', header=0, dtype='float', na_values=' NaN ')
    ## coerce data into DeepLabCut hierarchical format
    df['frame_index']=df.index
    df['scorer']=experimenter
    df = df.melt(id_vars=['frame_index','scorer'])
    new = df['variable'].str.rsplit("_",n=2,expand=True)
    df['variable'],df['cam'],df['coords'] = new[0], new[1], new[2]
    df=df.rename(columns={'variable':'bodyparts'})
    df['coords']=df['coords'].str.rstrip(" ").str.lower()
    if h_flip == True:
        df['value'][df['coords']=='x']= df['value'][df['coords']=='x'].apply(lambda x:width-1-x)
    df['bodyparts']=df['bodyparts'].str.lstrip(" ").astype("category")
    df['bodyparts'].cat.set_categories(markerlist,inplace=True)
    df['frame_index'] = ['labeled-data\\' + run+"Cam"+x[-1] + '\\img' + (f"{y:03d}") + '.png' for x, y in zip(df['cam'], df['frame_index'])]
    newdf = df.pivot_table(columns=['scorer', 'bodyparts', 'coords'],index='frame_index',values='value',aggfunc='first',dropna=False)
    newdf.index.name=None
    ## go into frame folders and get frame index ##
    extracted_frames = []
    for root, dirs, files in os.walk(labeled_data_path):
        for name in files:
            if name.endswith(".png") and run in root:
                camera_id = root.split(' ')[-1][-1]
                frame_no = int(name.split('.')[0].replace('img',''))
                new_name = 'labeled-data\\'+run+"Cam"+camera_id+'\\img' + (f"{frame_no:03d}") + '.png'
                extracted_frames.append(new_name)

    ## filter by list of extracted frames
    df_extracted = newdf.filter(items=pd.Index(extracted_frames),axis=0)

    ## split new df into cams 1 and 2
    df1 = df_extracted.filter(like=run+"Cam"+"1",axis=0)
    df2 = df_extracted.filter(like=run+"Cam"+"2",axis=0)

    ## split new df into cams 1 and 2, export as h5 and csv
    for x in [1,2]:
        cam_name = run+"Cam"+str(x)
        dfx = df_extracted.filter(like=cam_name,axis=0)
        data_name = labeled_data_path+cam_name+"\\CollectedData_"+experimenter+".h5"
        dfx.to_hdf(data_name, 'df_with_missing', format='table', mode='w')
        dfx.to_csv(data_name.split('.h5')[0]+'.csv')
        print("saved "+str(data_name))
        
run_names = ['c1_11Apr','c2_11Apr']
labeled_data_path = r"C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-separate\labeled-data\\"
csv_path = r"Z:\lab\NSF forelimb project\Phil_lab\C-arm\Ex\11Apr18.LaiRegnault.SEP101.LS.biceps_teres_lat\c1and2_11Apr_matched_xmalab2D.csv"
markerlist = ['Body_ds1_crn_cam1',
'Body_ds1_crn_cam2',
'Body_ds2_int_cam1',
'Body_ds2_int_cam2',
'Body_ds3_cdl_cam1',
'Body_ds3_cdl_cam2',
'Body_vn1_crn_cam1',
'Body_vn1_crn_cam2',
'Body_vn2_int_cam1',
'Body_vn2_int_cam2',
'Body_vn3_cdl_cam1',
'Body_vn3_cdl_cam2',
'Scapula_acr_cam1',
'Scapula_acr_cam2',
'Scapula_spi_cam1',
'Scapula_spi_cam2',
'Scapula_vtb_cam1',
'Scapula_vtb_cam2',
'Humerus_dpc_cam1',
'Humerus_dpc_cam2',
'Humerus_ent_cam1',
'Humerus_ent_cam2',
'Humerus_ect_cam1',
'Humerus_ect_cam2',
'Ulna_olc_cam1',
'Ulna_olc_cam2',
'Ulna_int_cam1',
'Ulna_int_cam2',
'Ulna_dst_cam1',
'Ulna_dst_cam2',
'Radius_prx_cam1',
'Radius_prx_cam2',
'Radius_int_cam1',
'Radius_int_cam2',
'Radius_dst_cam1',
'Radius_dst_cam2',
'Teres_maj_prx_cam1',
'Teres_maj_prx_cam2',
'Teres_maj_dst_cam1',
'Teres_maj_dst_cam2',
'Biceps_prx_cam1',
'Biceps_prx_cam2',
'Biceps_dst_cam1',
'Biceps_dst_cam2',]
for run in run_names:
    xmalab2dlc(run,csv_path,labeled_data_path, h_flip=True)


saved C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-separate\labeled-data\\c1_11AprCam1\CollectedData_Phil.h5
saved C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-separate\labeled-data\\c1_11AprCam2\CollectedData_Phil.h5
saved C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-separate\labeled-data\\c2_11AprCam1\CollectedData_Phil.h5
saved C:\Users\LabAdmin\Documents\Phil\DeepLabCut\dev\possum101_11Apr-Phil-2020-04-13-separate\labeled-data\\c2_11AprCam2\CollectedData_Phil.h5


#### 6. Convert XMALab exports to DeepLabCut format
No spaces in marker names, otherwise 2D export fails (more header columns than data)


In [ ]:
deeplabcut.analyze_videos(path_config_file,vids_to_analyze, videotype='.avi')

In [ ]:
deeplabcut.create_labeled_video(path_config_file,vids_to_analyze)

## Extract outlier frames [optional step]

This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. This step has many options, so please look at:

In [6]:
deeplabcut.extract_outlier_frames?

Signature: deeplabcut.extract_outlier_frames(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, outlieralgorithm='jump', comparisonbodyparts='all', epsilon=20, p_bound=0.01, ARdegree=3, MAdegree=1, alpha=0.01, extractionalgorithm='kmeans', automatic=False, cluster_resizewidth=30, cluster_color=False, opencv=True, savelabeled=True, destfolder=None)
Docstring:
   Extracts the outlier frames in case, the predictions are not correct for a certain video from the cropped video running from
   start to stop as defined in config.yaml.

   Another crucial parameter in config.yaml is how many frames to extract 'numframes2extract'.

   Parameter
   ----------
   config : string
       Full path of the config.yaml file as a string.

   videos : list
       A list of strings containing the full paths to videos for analysis or a path to the directory, where all the videos with same extension are stored.

   videotype: string, optional
       Checks for the extension of the video in case

In [ ]:
deeplabcut.extract_outlier_frames(path_config_file,vid_list) #pass a specific video

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset. 

In [5]:
%gui wx
deeplabcut.refine_labels(path_config_file)

Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Closing... The refined labels are stored in a subdirectory under labeled-data. Use the function 'merge_datasets' to augment the training dataset, and then re-train a network using creat

**NOTE:** Afterwards, if you want to look at the adjusted frames, you can load them in the main GUI by running: ``deeplabcut.label_frames(path_config_file)``

(you can add a new "cell" below to add this code!)

#### Once all folders are relabeled, check the labels again! If you are not happy, adjust them in the main GUI:

``deeplabcut.label_frames(path_config_file)``

Check Labels:

``deeplabcut.check_labels(path_config_file)``

In [6]:
#NOW, merge this with your original data:

deeplabcut.merge_datasets(path_config_file)

Merged data sets and updated refinement iteration to 1.
Now you can create a new training set for the expanded annotated images (use create_training_dataset).


## Create a new iteration of training dataset [optional step]
Following the refinement of labels and appending them to the original dataset, this creates a new iteration of training dataset. This is automatically set in the config.yaml file, so let's get training!

In [8]:
deeplabcut.create_training_dataset(path_config_file)

E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\training-datasets\iteration-1\UnaugmentedDataSet_1_3_4_5_unenhancedSep30  already exists!
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\dlc-models\iteration-1\1_3_4_5_unenhancedSep30-trainset95shuffle1  already exists!
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\dlc-models\iteration-1\1_3_4_5_unenhancedSep30-trainset95shuffle1//train  already exists!
E:\Users\Phil\DeepLabCut\dev\1_3_4_5_unenhanced-Phil-2019-09-30\dlc-models\iteration-1\1_3_4_5_unenhancedSep30-trainset95shuffle1//test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


In [ ]:
deeplabcut.train_network(path_config_file, maxiters=200000, displayiters=100)